In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame

In [114]:
df = pd.DataFrame(pd.read_csv('datasets/lab1/uci_malware_detection.csv'))

df

,Label,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,...,F_522,F_523,F_524,F_525,F_526,F_527,F_528,F_529,F_530,F_531
0,non-malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,non-malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,non-malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,non-malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,non-malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,1,0,1
369,malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
370,malicious,1,0,1,0,1,0,1,0,1,...,1,0,0,0,1,0,0,0,1,0
371,malicious,1,0,1,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0


## Розділяємо датасет на тренувальний та тестувальний, застосовуємо логістичну регресію

In [115]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

x = df.drop('Label', axis=1)
y = LabelEncoder().fit_transform(df['Label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

## Застосовуємо логістичну регресію

In [116]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression()
logreg_model.fit(x_train, y_train)

y_pred = logreg_model.predict(x_test)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

Matrix:
[[55  0]
 [ 1 19]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        55
           1       1.00      0.95      0.97        20

    accuracy                           0.99        75
   macro avg       0.99      0.97      0.98        75
weighted avg       0.99      0.99      0.99        75

Accuracy: 0.9866666666666667


# Lab 2: Дерево рішень та ліс дерев рішень

## Dataset #1: Emails

In [ ]:
import re
from nltk.corpus import stopwords
import string

df_2 = pd.DataFrame(pd.read_csv('datasets/lab2/Phishing_Email.csv')) # https://www.kaggle.com/datasets/subhajournal/phishingemails
df_2.dropna(inplace=True)
df_2.drop(columns=df_2.columns[0], inplace=True)
def process_text(transformer: str) -> str:
    transformer = transformer.lower()
    transformer = re.sub(r"https*\S+", " ", transformer)
    transformer = re.sub(r"^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-]+)(\.[a-zA-Z]{2,5}){1,2}$", " ", transformer)
    transformer = re.sub("[%s]" % re.escape(string.punctuation), " ", transformer)
    transformer = re.sub(r"\s{2,}", " ", transformer)
    
    result = ' '.join([word for word in transformer.split(' ') if word not in stopwords.words("english")])
    return result.strip()

x = df_2.drop('Email Type', axis=1)['Email Text'].apply(process_text)
y = LabelEncoder().fit_transform(df_2['Email Type'])


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

## Векторизація тексту за TF-IDF та Count Vectorizer

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv = CountVectorizer(max_features=1000, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

x_train_vectorised_tfidf = tfidf.fit_transform(x_train)
x_test_vectorised_tfidf = tfidf.transform(x_test)
x_train_vectorised_cv = cv.fit_transform(x_train)
x_test_vectorised_cv = cv.transform(x_test)

## Decision Tree

In [68]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth=5, random_state=0, criterion='gini')
decision_tree.fit(x_train_vectorised_cv, y_train)

y_pred = decision_tree.predict(x_test_vectorised_cv)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

Matrix:
[[ 645  812]
 [  73 2197]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.44      0.59      1457
           1       0.73      0.97      0.83      2270

    accuracy                           0.76      3727
   macro avg       0.81      0.71      0.71      3727
weighted avg       0.80      0.76      0.74      3727

Accuracy: 0.7625436007512745


## Random Forest

In [72]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=5, random_state=0, criterion='gini', n_estimators=100)
random_forest.fit(x_train_vectorised_tfidf, y_train)

y_pred = random_forest.predict(x_test_vectorised_tfidf)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

Matrix:
[[ 786  671]
 [  19 2251]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.54      0.69      1457
           1       0.77      0.99      0.87      2270

    accuracy                           0.81      3727
   macro avg       0.87      0.77      0.78      3727
weighted avg       0.85      0.81      0.80      3727

Accuracy: 0.8148645022806547


## Dataset #2: Access behavior anomaly

In [111]:
from sklearn.preprocessing import StandardScaler

df_3 = pd.read_csv('datasets/lab2/supervised_dataset.csv') # https://www.kaggle.com/datasets/tangodelta/api-access-behaviour-anomaly-dataset?select=supervised_dataset.csv

df_3.drop(df_3.columns[0], inplace=True, axis=1)
df_3.drop('_id', inplace=True, axis=1)
df_3.drop('source', inplace=True, axis=1)
df_3.dropna(inplace=True)

le = LabelEncoder()
df_3['ip_type'] = le.fit_transform(df_3['ip_type'])
df_3['classification'] = le.fit_transform(df_3['classification'])
x = df_3.drop('classification', axis=1)
y = df_3['classification']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
scaler.fit(x_train)

scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(x_test)

## Decision Tree and Random Forest

In [112]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()

decision_tree.fit(scaled_x_train, y_train)
y_pred = decision_tree.predict(scaled_x_test)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

random_forest.fit(scaled_x_train, y_train)
y_pred = random_forest.predict(scaled_x_test)

print(f'Matrix:\n{confusion_matrix(y_test, y_pred)}\nClassification Report:\n{classification_report(y_test, y_pred)}\nAccuracy: {accuracy_score(y_test, y_pred)}')

Matrix:
[[218   0]
 [  0 121]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       218
           1       1.00      1.00      1.00       121

    accuracy                           1.00       339
   macro avg       1.00      1.00      1.00       339
weighted avg       1.00      1.00      1.00       339

Accuracy: 1.0
Matrix:
[[218   0]
 [  0 121]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       218
           1       1.00      1.00      1.00       121

    accuracy                           1.00       339
   macro avg       1.00      1.00      1.00       339
weighted avg       1.00      1.00      1.00       339

Accuracy: 1.0
